In [2]:
import pandas as pd 
import os
import glob
import difflib

In [3]:
n=1

mdf=pd.read_excel(f'/workspaces/vikrant_dubai/RAW/FUJ/RAW/FUJ {n}.xlsx')
excel_file = pd.ExcelFile(f'/workspaces/vikrant_dubai/RAW/FUJ/RAW/FUJ {n}.xlsx')

df = pd.DataFrame(columns=["Station Name", "Latitude", "Longitude", "Region", "Area Classification"])
dfs = []
for i in range(len(excel_file.sheet_names)-5):
    df = pd.read_excel(f"/workspaces/vikrant_dubai/RAW/FUJ/RAW/FUJ {n}.xlsx", sheet_name = excel_file.sheet_names[i])
    df=df.iloc[:,:9]


    mapped_data = {
    "Station Name": df.iloc[0, 2],  
    "Latitude": df.iloc[1, 3],  
    "Longitude": df.iloc[1, 5],  
    "Region": df.iloc[0, 5],  
    "Area Classification": df.iloc[0, 8], 
    # "Emission Source": df.iloc[0, 8],
    "Station Operation Year": pd.to_datetime(df.iloc[1, 8]).year
    }

    temp_df = pd.DataFrame([mapped_data])
    dfs.append(temp_df)

final_df = pd.concat(dfs, ignore_index=True)

final_df.to_csv(f'/workspaces/vikrant_dubai/SQL/STdetails.csv', index=False)

In [76]:
mdf=pd.read_excel('/workspaces/vikrant_dubai/RAW/FUJ/RAW/FUJ 1.xlsx')
excel_file = pd.ExcelFile('/workspaces/vikrant_dubai/RAW/FUJ/RAW/FUJ 1.xlsx')

columns_of_main_df = ['Station Name','Date', 'Time', 'SO2', 'Data Cap. (SO2)', 'SO2 Avg', 'NO2', 'Data Cap. (NO2)', 'NO2 Avg',      'O3', 'Data Cap. (O3)', 'O3 8H Avg.', 'Daily Capture O3', 'O3 Daily Average', 'O3 Daily Average (NS)', 'CO',  'Data Cap. (CO)', 'CO 8H Avg.', 'Daily Capture CO', 'CO Daily Average', 'PM10',    'Data Cap. (PM10)', 'PM10 Avg', 'PM2.5', 'Data Cap. (PM2.5)', 'PM2.5 Avg', 'Lower Ambient Temperature', 'Upper Ambient Temperature', 'Barometric Pressure', 'Relative Humidity', 'Wind Direction', 'Wind Speed', 'H2S', 'Toluene', 'O-Xylene', 'Ethylbenzene', 'MP-Xylene', 'Benzene', 'CH4', 'NMHC', 'THC', 'Noise', 'Solar Radiation','Atm P.']
for s in range(len(excel_file.sheet_names)-5):
    main_df = pd.DataFrame(columns=columns_of_main_df)

    # Read the Excel file and preprocess the data
    mdf = pd.read_excel('/workspaces/vikrant_dubai/RAW/FUJ/RAW/FUJ 1.xlsx', sheet_name=excel_file.sheet_names[s])
    mdf = mdf.drop([0,1,2,3,4]).reset_index(drop=True)
    mdf.columns = mdf.iloc[0]
    mdf = mdf.drop([0,1,2]).reset_index(drop=True)
    mdf = mdf.iloc[:, 1:]


    # Check if 'Date/Time' column exis
    mdf['Date/Time'] = pd.to_datetime(mdf['Date/Time'], format='%d/%m/%Y %H:%M', dayfirst=True) 

    mdf['Date'] = mdf['Date/Time'].dt.date
    mdf['Time'] = mdf['Date/Time'].dt.time



    new_df_columns=mdf.columns 

    # Convert new_df_columns to a list to allow mutable operations
    new_df_columns = list(new_df_columns)

    # Filter out non-string values from new_df_columns
    new_df_columns = [col if isinstance(col, str) else '' for col in new_df_columns]

    # Iterate over the series and replace the column names
    for col_index in range(1, len(new_df_columns)):
        if 'DATA CAP.' in new_df_columns[col_index]:
            if new_df_columns[col_index-1] == 'DATA COUNT':
                new_df_columns[col_index] = f"DATA CAP. ({new_df_columns[col_index-2]})"
            else:
                new_df_columns[col_index] = f"DATA CAP. ({new_df_columns[col_index-1]})"


    mdf.columns = new_df_columns

    # add column station name
    mdf['Station Name'] = excel_file.sheet_names[s]

    # Format the 'Date' column
    mdf['Date'] = pd.to_datetime(mdf['Date']).dt.strftime('%d/%m/%Y')




    # List of values to find matches for
    values = [str(col).upper() for col in mdf.columns]

    # List of column names
    l1 = [str(col).upper() for col in main_df.columns]


    # Loop through values and find the closest match
    matches = {}
    for value in values:
        closest_match = difflib.get_close_matches(value, l1, n=1)
        if closest_match:
            main_df[main_df.columns[main_df.columns.str.upper() == closest_match[0]][0]] = mdf[mdf.columns[mdf.columns.str.upper() == value][0]]
        else:
            main_df[value] = ''



    main_df.to_csv(f'/workspaces/vikrant_dubai/Transformed/{excel_file.sheet_names[s]}.csv', index=False)
            
# Get a list of all CSV files in the transformed folder
csv_files = [f for f in os.listdir('/workspaces/vikrant_dubai/Transformed') if f.endswith('.csv')]

# Read and concatenate all CSV files
combined_df = pd.concat([pd.read_csv(os.path.join('/workspaces/vikrant_dubai/Transformed', f)) for f in csv_files], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv(f'/workspaces/vikrant_dubai/final/FUJ/FUJ 1.csv', index=False)


# Get a list of all files in the Transformed folder
files = glob.glob('/workspaces/vikrant_dubai/Transformed/*')

# Iterate over the list of files and remove each one
for f in files:
    os.remove(f)